In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook

In [2]:

dpdata = pd.read_excel(r'C:\Users\UNOCC-data\Downloads\DiplomaticPulseIsraelPalestineFrom07Oct2023.xlsx', sheet_name='Sheet1')

In [3]:
def clean_text(text):
    if text:
        # Replace tab characters with spaces
        text = text.replace('\t', ' ')
        
        # Split the text into lines, remove empty lines, and strip leading/trailing whitespaces
        cleaned_lines = [line.strip() for line in text.split('\n') if line.strip()]
        cleaned_text = ' '.join(cleaned_lines)
        return cleaned_text
    else:
        return text

In [7]:
dpdata['cleaned_statement'] = dpdata['statement'].apply(clean_text)
#dpdata.iloc[1]['cleaned_statement']

In [8]:
from googletrans import Translator
from langdetect import detect


def translate_text(text, target_lang='en'):
    #print (len(text))
    if text is not None:
        if len(text) <= 5000:
            source_lang = detect(text)
            
            if source_lang:
                translator = Translator()
                translated = translator.translate(text, src=source_lang, dest=target_lang)
                return translated.text
        elif len(text) >5000:
            source_lang = detect(text)
            if source_lang:
                translator = Translator()
                chunk_size =4500
                # Split the text into chunks of the specified size
                text_chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

                # Translate each chunk and store the translations
                translated_chunks = [translator.translate(chunk, src=source_lang, dest=target_lang).text for chunk in text_chunks]

                # Combine the translated chunks to get the full translated text
                translated_text = ' '.join(translated_chunks)
                return translated_text

    return text    # Return the original text for None or empty values


In [9]:
dpdata['translated_statement'] = dpdata.apply(lambda row: translate_text(row['cleaned_statement']) if row['language'] != 'English' else row['cleaned_statement'], axis=1)

In [36]:
#dpdata['translated_statement']

0      The EU foreign ministers affirmed their comple...
1      I acknowledge the traditional owners of this l...
2      On 9 and 10 October 2023, the Ambassador-at-La...
3      Thank you High Commissioner for the presentati...
4      United States Mission to the United Nations Of...
                             ...                        
261    The White House 1600 Pennsylvania Ave NW Washi...
262    The current situation of Malaysian citizens in...
263    Israel has the right to defend itself, and not...
264    Home & GT;News & GT;Foreign Ministry receives ...
265    I know that this is the last of your big tour ...
Name: translated_statement, Length: 266, dtype: object

In [10]:
writer = pd.ExcelWriter('translated-Israel-oPt.xlsx', engine='openpyxl')
dpdata.to_excel(writer, sheet_name='Main', index=False)
writer.book.save('translated-Israel-oPt.xlsx')